In [1]:
!pip install tensorflow-federated==0.19.0

     |████████████████████████████████| 602 kB 15.5 MB/s 
     |████████████████████████████████| 192 kB 53.1 MB/s 
     |████████████████████████████████| 460.3 MB 7.2 kB/s 
     |████████████████████████████████| 172 kB 51.7 MB/s 
     |████████████████████████████████| 132 kB 41.3 MB/s 
     |████████████████████████████████| 14.8 MB 45.3 MB/s 
     |████████████████████████████████| 462 kB 55.4 MB/s 
     |████████████████████████████████| 1.2 MB 42.4 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=558539567d310e8acd378eed1a9300a6f12c32b80b8fda42e7f2e0bde2271ae6
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68715 sha256=45a4b5f3807ec8611248eca9b93b883a8f809320bea383c9ac7190399b910143
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Succes

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff

csv_url = "https://docs.google.com/spreadsheets/d/1eJo2yOTVLPjcIbwe8qSQlFNpyMhYj-xVnNVUTAhwfNU/gviz/tq?tqx=out:csv"

df = pd.read_csv(csv_url, na_values=("?",))

client_id_colname = 'native.country' # the column that represents client ID
SHUFFLE_BUFFER = 1000
NUM_EPOCHS = 1

In [3]:
# split client id into train and test clients
client_ids = df[client_id_colname].unique()
train_client_ids = pd.DataFrame(client_ids).sample(frac=0.5).values.tolist()

[['Peru'], ['Iran'], ['Nicaragua'], ['Outlying-US(Guam-USVI-etc)'], ['India'], ['Jamaica'], ['Japan'], ['Trinadad&Tobago'], ['Yugoslavia'], ['France'], ['Holand-Netherlands'], ['Portugal'], ['Columbia'], ['South'], ['United-States'], [nan], ['Thailand'], ['Vietnam'], ['Puerto-Rico'], ['Italy'], ['England']]


In [15]:
test_client_ids = [x for x in client_ids if x not in train_client_ids]

['United-States', nan, 'Mexico', 'Greece', 'Vietnam', 'China', 'Taiwan', 'India', 'Philippines', 'Trinadad&Tobago', 'Canada', 'South', 'Holand-Netherlands', 'Puerto-Rico', 'Poland', 'Iran', 'England', 'Germany', 'Italy', 'Japan', 'Hong', 'Honduras', 'Cuba', 'Ireland', 'Cambodia', 'Peru', 'Nicaragua', 'Dominican-Republic', 'Haiti', 'El-Salvador', 'Hungary', 'Columbia', 'Guatemala', 'Jamaica', 'Ecuador', 'France', 'Yugoslavia', 'Scotland', 'Portugal', 'Laos', 'Thailand', 'Outlying-US(Guam-USVI-etc)']


In [69]:
def create_tf_dataset_for_client_fn(client_id):
  # a function which takes a client_id and returns a
  # tf.data.Dataset for that client
  # print(client_id)
  client_data = df[df[client_id_colname] == client_id[0]]
  # print(client_data)
  dataset = tf.data.Dataset.from_tensor_slices(client_data.fillna('').to_dict("list"))
  dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(1).repeat(NUM_EPOCHS)
  return dataset

In [71]:
train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
        client_ids=train_client_ids,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )
test_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
        client_ids=test_client_ids,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )

In [72]:
example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

print(type(example_dataset))
example_element = iter(example_dataset).next()
print(example_element)
# <class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>
# {'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([37], dtype=int32)>, 'workclass': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Local-gov'], dtype=object)>, ...

<class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>
{'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([48], dtype=int32)>, 'workclass': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Private'], dtype=object)>, 'fnlwgt': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([280422], dtype=int32)>, 'education': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Some-college'], dtype=object)>, 'education.num': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([10], dtype=int32)>, 'marital.status': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Separated'], dtype=object)>, 'occupation': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Other-service'], dtype=object)>, 'relationship': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Not-in-family'], dtype=object)>, 'race': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'White'], dtype=object)>, 'sex': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Female'], dtype=object)>, 'capital.gain': <tf.Ten